In [5]:
import pandas as pd

filepath = r"https://github.com/data-to-insight/ERN-sessions/raw/main/data/903_xlsx_errors.xlsx"
 
df = pd.read_excel(filepath, sheet_name=None)

In [6]:
df

{'ad1':         CHILD         DOB    DATE_INT  DATE_MATCH  FOSTER_CARE NB_ADOPTR  \
 0      988635  15/06/2013  22/11/2015  22/11/2015            1         2   
 1      161125  24/07/2002  08/05/2015  08/05/2015            1         1   
 2      590545  24/11/2008  12/09/2015  12/09/2015            1         2   
 3      757606  12/12/2003  16/09/2015  16/09/2015            0         1   
 4      921450  11/10/2006  02/03/2015  02/03/2015            0         1   
 5        4022  11/05/2001  11/11/2015  11/11/2015            1         1   
 6      127650  24/02/2013  05/09/2015  05/09/2015            0         1   
 7      624227  04/07/2008  27/01/2015  27/01/2015            0         2   
 8      711458  06/05/2009  19/03/2015  19/03/2015            0         1   
 9      549562  25/07/2009  30/10/2015  30/10/2015            1         1   
 10      83008  05/06/2005  06/02/2015  06/02/2015            1         2   
 11     764585  01/06/2011  17/05/2015  17/05/2015            0      